## Basic import data to test

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
import pickle

# import data
print('start reading')
f = open("TraData.csv")
data = np.loadtxt(f, delimiter=',')

# select data
x = data[:, 1:]  # select columns 1 through end
y = data[:, 0]   # select column 0, the result
print('done separate x and y')


start reading
done separate x and y


## Data preprocess
- first normalize data and scaling 
- then select 95 % important thing out of the data 
    - select percentile
- Use PCA to take data
- disjoint training and validation dataset

In [3]:
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA
from sklearn import preprocessing
from scipy.stats import pearsonr
import numpy as np

normalizer = preprocessing.Normalizer().fit(x)
X_scaled = preprocessing.RobustScaler().fit(x)
normalizer.transform(x)
X_scaled.transform(x)
print(x.shape)

X_new = SelectPercentile(percentile=95).fit_transform(x, y)
print(X_new.shape)

pca = PCA(n_components=29)
X_pca = pca.fit_transform(X_new)
print(X_pca.shape)
# print(pca.explained_variance_ratio_)

train_x, valid_x,train_y,valid_y = train_test_split(X_pca,y,test_size=0.1)
print('start2')


(500000, 31)
(500000, 29)
(500000, 29)
start2


## Random Forest

In [4]:
from sklearn.ensemble import RandomForestClassifier

print('start random forest')
# random forest
clf2 = RandomForestClassifier(criterion='entropy',max_samples = 10,
                                  n_estimators=30)
clf2 = clf2.fit(train_x,train_y)

print('Training Acc:\n',clf2.score(train_x,train_y))
print('Training data predict:\n',clf2.predict(train_x))
print('Validate Accuracy:\n',clf2.score(valid_x,valid_y))
print('Validate data predict:\n',clf2.predict(valid_x))

start random forest
Training Acc:
 0.5645955555555555
Training data predict:
 [1. 1. 1. ... 0. 1. 1.]
Validate Accuracy:
 0.56436
Validate data predict:
 [0. 1. 1. ... 1. 0. 1.]


## Neuro network

In [5]:
from sklearn.neural_network import MLPClassifier

clf4 = MLPClassifier(activation='relu',solver='sgd',max_iter=2000,learning_rate_init=0.001,
                    hidden_layer_sizes=(45,35)) 
print('start neuro network')
clf4 = clf4.fit(train_x,train_y)
print('Training Acc:\n',clf4.score(train_x,train_y))
print('Training data predict:\n',clf4.predict(train_x))
print('Validate Accuracy:\n',clf4.score(valid_x,valid_y))
print('Validate data predict:\n',clf4.predict(valid_x))


start neuro network
